In [1]:
!python --version

Python 3.11.11


In [2]:
!pip list

Package                                 Version
--------------------------------------- --------------
absl-py                                 2.1.0
accelerate                              0.34.2
adagio                                  0.2.6
aiobotocore                             2.19.0
aiohttp                                 3.9.5
aiohttp-cors                            0.7.0
aioitertools                            0.12.0
aiosignal                               1.3.2
aiosqlite                               0.19.0
alembic                                 1.14.1
altair                                  5.5.0
amazon-q-developer-jupyterlab-ext       3.4.6
amazon_sagemaker_jupyter_ai_q_developer 1.0.16
amazon_sagemaker_jupyter_scheduler      3.1.9
amazon-sagemaker-sql-editor             0.1.14
amazon-sagemaker-sql-execution          0.1.6
amazon-sagemaker-sql-magic              0.1.3
annotated-types                         0.7.0
ansi2html                               1.9.2
ansicolors      

In [3]:
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

In [4]:
!pip install --disable-pip-version-check -q awscli==1.18.216 boto3==1.16.56 botocore==1.19.56

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
jupyter-ai 2.29.1 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.19.0 requires botocore<1.36.4,>=1.36.0, but you have botocore 1.19.56 which is incompatible.
amazon-sagemaker-sql-editor 0.1.14 requires boto3<2,>=1.29.6, but you have boto3 1.16.56 which is incompatible.
amazon-sagemaker-sql-editor 0.1.14 requires botocore<2,>=1.31.64, but you have botocore 1.19.56 which is incompatible.
amazon-sagemaker-sql-execution 0.1.6 requires boto3>=1.28.64, but you have boto3 1.16.56 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.
autogluon-multimodal 1.2 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 whic

In [5]:
!pip install --disable-pip-version-check -q sagemaker==2.29.0
!pip install --disable-pip-version-check -q smdebug==1.0.1
!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26

In [6]:

!conda install -y pytorch==1.6.0 -c pytorch

Channels:
 - pytorch
 - conda-forge
Platform: linux-64
Solving environment: | ^C


In [7]:
!pip install imblearn

In [8]:
!pip install time

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [9]:
!pip install --disable-pip-version-check -q tensorflow==2.8.1

ERROR: Could not find a version that satisfies the requirement tensorflow==2.8.1 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow==2.8.1


In [10]:
!pip install --disable-pip-version-check -q transformers==3.5.1

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [2 lines of output]
      /bin/sh: 1: pkg-config: not found
      Failed to find sentencepiece pkgconfig
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [11]:
!pip install --disable-pip-version-check -q torchserve==0.3.0
!pip install --disable-pip-version-check -q torch-model-archiver==0.3.0

In [12]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0

^C
ERROR: Operation cancelled by user


In [13]:

!pip install --disable-pip-version-check -q SQLAlchemy==1.3.22

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-scheduler 2.10.0 requires sqlalchemy<3,>=2.0, but you have sqlalchemy 1.3.22 which is incompatible.
langchain 0.3.17 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.3.22 which is incompatible.
langchain-community 0.3.16 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.3.22 which is incompatible.
mlflow 2.20.1 requires sqlalchemy<3,>=1.4.0, but you have sqlalchemy 1.3.22 which is incompatible.
optuna 4.2.0 requires sqlalchemy>=1.4.2, but you have sqlalchemy 1.3.22 which is incompatible.
sagemaker-mlflow 0.1.0 requires boto3>=1.34, but you have boto3 1.16.56 which is incompatible.


In [14]:
!pip install --disable-pip-version-check -q awswrangler==2.15.0

ERROR: Ignored the following yanked versions: 3.2.0
ERROR: Could not find a version that satisfies the requirement awswrangler==2.15.0 (from versions: 0.0b0, 0.0b2, 0.0b3, 0.0b4, 0.0b5, 0.0b6, 0.0b7, 0.0b8, 0.0b9, 0.0b10, 0.0b11, 0.0b12, 0.0b13, 0.0b14, 0.0b15, 0.0b16, 0.0b17, 0.0b18, 0.0b19, 0.0b20, 0.0b21, 0.0b22, 0.0b23, 0.0b24, 0.0b25, 0.0b26, 0.0b27, 0.0b28, 0.0b29, 0.0b30, 0.0b31, 0.0b32, 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.0.8, 0.0.9, 0.0.10, 0.0.11, 0.0.12, 0.0.13, 0.0.14, 0.0.15, 0.0.16, 0.0.17, 0.0.18, 0.0.19, 0.0.20, 0.0.21, 0.0.22, 0.0.23, 0.0.24, 0.0.25, 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.5, 0.2.6, 0.3.0, 0.3.1, 0.3.2, 2.18.0, 2.19.0, 2.20.0, 2.20.1, 3.0.0rc1, 3.0.0rc2, 3.0.0rc3, 3.0.0, 3.1.0, 3.1.1, 3.2.1, 3.3.0, 3.4.0, 3.4.1, 3.4.2, 3.5.0, 3.5.1, 3.5.2, 3.6.0, 3.7.0, 3.7.1, 3.7.2, 3.7.3, 3.8.0, 3.9.0, 3.9.1, 3.10.0, 3.10.1, 3.11.0)
ERROR: No matching distribution found for awswrangler==2.15.0


In [15]:
!pip install --disable-pip-version-check -q stepfunctions==2.0.0rc1

In [16]:
!conda install -y zip

Channels:
 - conda-forge
Platform: linux-64
failed

CondaError: KeyboardInterrupt



In [ ]:
!pip install --disable-pip-version-check -q matplotlib==3.1.3

In [ ]:
!pip install --disable-pip-version-check -q seaborn==0.10.0

In [ ]:
!pip install protobuf==3.20.*
!python --version

In [ ]:
!pip list

In [ ]:
setup_dependencies_passed = True

In [ ]:
%store setup_dependencies_passed

In [ ]:
%store

In [ ]:
!pip install --disable-pip-version-check -q sagemaker==2.29.0
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [ ]:

setup_s3_bucket_passed = False

In [ ]:
print("Default bucket: {}".format(bucket))

In [ ]:
%%bash

aws s3 ls s3://${bucket}/

In [ ]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

In [ ]:
%store setup_s3_bucket_passed

In [ ]:
%store

In [ ]:
%store

In [1]:
import boto3
import sagemaker
import time
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})

iam = boto3.client("iam", config=config)

In [4]:
role_name = role.split("/")[-1]

print("Role name: {}".format(role_name))

Role name: LabRole


In [5]:
setup_iam_roles_passed = False

In [6]:
admin = False
post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        break

setup_iam_roles_passed = True
print("[OK] You are all set up to continue with this workshop!")

[OK] You are all set up to continue with this workshop!


In [7]:
%store setup_iam_roles_passed

Stored 'setup_iam_roles_passed' (bool)


In [8]:
%store

Stored variables and their in-db values:
s3_private_path_tsv                   -> 's3://sagemaker-us-east-1-373893336492/inspections
s3_public_path_tsv                    -> 's3://usd-mads-508/inspections-pds/tsv'
setup_dependencies_passed             -> True
setup_iam_roles_passed                -> True
setup_s3_bucket_passed                -> True


In [9]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [1]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>